# DIVAnd in sigma coordinates
### Import packages

In [ ]:
using NCDatasets
using DataStructures
using DIVAnd
using Dates
using Statistics
using Random
using Printf
using PyPlot
using Interpolations
using Roots
include("../config.jl")

## Configuration
* Define the horizontal, vertical (depth levels) and temporal resolutions.
* Select the variable of interest

In [ ]:
dx, dy = 0.125, 0.125
lonr = 11.5:dx:20
latr = 39:dy:46

depthr = [0.,5., 10., 15., 20., 25., 30., 40., 50., 66, 
    75, 85, 100, 112, 125, 135, 150, 175, 200, 225, 250, 
    275, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 
    800, 850, 900, 950, 1000, 1050, 1100, 1150, 1200, 1250, 
    1300, 1350, 1400, 1450, 1500, 1600, 1750, 1850, 2000];
depthr = depthr[1:20];

## 1. Read your ODV file
Adapt the `datadir` and `datafile` values.     
The example is based on a sub-setting of the [Mediterranean Sea](https://www.seadatanet.org/Products#/metadata/cd552057-b604-4004-b838-a4f73cc98fcf) aggregated dataset.      
The dataset has been extracted around the Adriatic Sea and exported to a netCDF using [Ocean Data View](https://odv.awi.de/).
<img src="./Images/MedSeaAggreg.jpg" width="450px">

Download the data files (test and full) if needed.

In [ ]:
datafile = adriaticfile
download_check(datafile, adriaticfileURL)

Read the full file:

In [ ]:
@time obsval,obslon,obslat,obsdepth,obstime,obsid = NCODV.load(Float64, datafile, 
    "Water body salinity");

In [ ]:
figure("Adriatic-Data")
ax = subplot(1,1,1)
plot(obslon, obslat, "ko", markersize=.1, markerfacecolor="k")
aspectratio = 1/cos(mean(latr) * pi/180)
ax.tick_params("both",labelsize=6)
gca().set_aspect(aspectratio)

Check the extremal values of the observations

In [ ]:
checkobs((obslon,obslat,obsdepth,obstime),obsval,obsid)

## 2. Extract the bathymetry
It is used to delimit the domain where the interpolation is performed.

### 2.1 Choice of bathymetry
Modify `bathname` according to the resolution required.

In [ ]:
bathname = gebco08file
download_check(bathname, gebco08fileURL)

In [ ]:
@time bx,by,b = load_bath(bathname,true,lonr,latr);

In [ ]:
figure("Adriatic-Bathymetry")
ax = subplot(1,1,1)
pcolor(bx, by, permutedims(b, [2,1]));
colorbar(orientation="vertical", shrink=0.8).ax.tick_params(labelsize=8)
contour(bx, by, permutedims(b, [2,1]), [0, 0.1], colors="k", linewidths=.5)
gca().set_aspect(aspectratio)
ax.tick_params("both",labelsize=6)

### 2.2 Create mask
* False for sea        
* True for land

In [ ]:
mask = falses(size(b,1),size(b,2),length(depthr))
for k = 1:length(depthr)
    for j = 1:size(b,2)
        for i = 1:size(b,1)
            mask[i,j,k] = b[i,j] >= depthr[k]
        end
    end
end
@show size(mask)

In [ ]:
figure("Adriatic-Mask")
ax = subplot(1,1,1)
gca().set_aspect(aspectratio)
ax.tick_params("both",labelsize=6)
ax.pcolor(bx, by, transpose(mask[:,:,1]), cmap=PyPlot.cm.binary_r)
ax.set_title("Land-sea mask")

### 2.3 Edit the mask
As an example we will remove the Tyrrhenian Sea from the domain.

In [ ]:
grid_bx = [i for i in bx, j in by];
grid_by = [j for i in bx, j in by];

In [ ]:
mask_edit = copy(mask);
sel_mask1 = (grid_by .<= 42.6) .& (grid_bx .<= 14.);
sel_mask2 = (grid_by .<= 41.2) .& (grid_bx .<= 16.2);
mask_edit = mask_edit .* .!sel_mask1 .* .!sel_mask2;
@show size(mask_edit)

In [ ]:
#sel = (obslon .<  maximum(bx)) .& (obslon .>  minimum(bx)) .& (obslat .<  maximum(by)) .& (obslat .>  minimum(bx))
#obslon=obslon[sel]
#obslat=obslat[sel]
#obsdepth=obsdepth[sel]
#obsval=obsval[sel]

The edited mask now looks like this:

In [ ]:
figure("Adriatic-Mask-Edited")
ax = subplot(1,1,1)
ax.tick_params("both",labelsize=6)
ax.pcolor(bx, by, mask_edit[:,:,1]', cmap=cmap=PyPlot.cm.binary_r)
ax.set_title("Edited land-sea mask")
gca().set_aspect(aspectratio)

## Define a vertical correlation length function in real space

In [ ]:
function Lvert(x,y,z)
    return 5.0 + z/5.0
end

LHOR=1.5

eps2=0.1

iscyclic=falses(3)

## 3. Analysis in Cartesian Coordinates (degrees for simplicity)

In [ ]:
xc,yc,zc=ndgrid(lonr,latr,depthr)
pmc = ones(size(xc)) / (xc[2,1,1]-xc[1,1,1]);
pnc = ones(size(xc)) / (yc[1,2,1]-yc[1,1,1]);
poc = ones(size(xc)) / (zc[1,1,2]-zc[1,1,1]);


In [ ]:
sz = (length(lonr),length(latr),length(depthr));
lenx = fill(LHOR,sz)  
leny = lenx./aspectratio
lenz = Lvert.(xc,yc,zc);
@show mean(lenx),mean(leny),mean(lenz)
epsilon2 = eps2;
#epsilon2 = epsilon2 * rdiag;

In [ ]:
obsmean=mean(obsval)

fi,s=DIVAndrun(mask_edit,(pmc,pnc,poc),(xc,yc,zc),(obslon,obslat,obsdepth),obsval.-obsmean,(lenx,leny,lenz),epsilon2);

In [ ]:
jcut=30
pcolor(xc[:,jcut,:],-zc[:,jcut,:],fi[:,jcut,:].+obsmean),colorbar()

In [ ]:
pcolor(xc[:,:,1],yc[:,:,1],fi[:,:,1])

In [ ]:
kcut=15
pcolor(xc[:,:,kcut],yc[:,:,kcut],fi[:,:,kcut])

In [ ]:
@show zc[1,1,kcut]

# 4.  Coordinate change

In [ ]:
bfun=linear_interpolation((bx,by),b;extrapolation_bc=0.0)

function ztosigma(x,y,z)
    return z/max(2.0,bfun(x,y))
end
function sigmatoz(x,y,sigma)
    return sigma*max(2.0,bfun(x,y))
end
# For more general cases it is easy to make the z(sigma) function more general, but then the inverse will need some
# search of zeros. Needed to get fracional indexes and horizontal plots

### Vertical position in sigma space

In [ ]:
obssigma=ztosigma.(obslon,obslat,obsdepth)

In [ ]:
xs,ys,sigma=ndgrid(lonr,latr,range(0;stop=1,length=25))
# vertical position in real space
zs=sigmatoz.(xs,ys,sigma)
pms = ones(size(xs)) / (xs[2,1,1]-xs[1,1,1]);
pns = ones(size(xs)) / (ys[1,2,1]-ys[1,1,1]);
pos = ones(size(xs)) / (sigma[1,1,2]-sigma[1,1,1]);
sz=size(xs)
masks=trues(size(xs))
masks[:,:,:].=mask_edit[:,:,1]

lenx = fill(LHOR,sz)  
leny = lenx./aspectratio
lenz = Lvert.(xs,ys,zs);    

# Since you work in sigma space:
lenz=lenz./max.(2.0,bfun.(xs,ys))
    
epsilon2 = eps2;
#epsilon2 = epsilon2 * rdiag;
obsmean=mean(obsval)

fi,s=DIVAndrun(masks,(pms,pns,pos),(xs,ys,sigma),(obslon,obslat,obssigma),obsval.-obsmean,(lenx,leny,lenz),epsilon2);


In [ ]:
jcut=30
pcolor(xs[:,jcut,:],-sigma[:,jcut,:],fi[:,jcut,:].+obsmean),colorbar()

In [ ]:
jcut=30
pcolor(xs[:,jcut,:],-zs[:,jcut,:],fi[:,jcut,:].+obsmean),colorbar()

## Now exploiting directly non-uniform grids by creating the pmn etc

In [ ]:
# We already have the z positions of the sigma layers we want to work with from above
# So now just recalculate the metrics:
pozc=deepcopy(pos)
for i = 1:sz[3]
        i0 = max(i - 1, 1)
        i1 = min(i + 1, sz[3])
        pozc[:,:,i].=(i1-i0)./(zs[:,:,i1].-zs[:,:,i0])
end
# Or simpler divide sigma pos  b: No the former is more general in case you have different stretching depending on depth
# pocz=pos./max.(2.0,bfun.(xs,ys));



### Get fractional indexes by working in sigma space:

In [ ]:
Igrid = localize_separable_grid((obslon,obslat,obssigma), masks, (xs,ys,sigma), iscyclic)

In [ ]:
masks[:,:,:].=mask_edit[:,:,1]


lenx = fill(LHOR,sz)  
leny = lenx./aspectratio
lenz = Lvert.(xs,ys,zs);

epsilon2 = eps2;
#epsilon2 = epsilon2 * rdiag;
obsmean=mean(obsval)

fi,s=DIVAndrun(masks,(pms,pns,pozc),(xs,ys,zs),(obslon,obslat,obsdepth),obsval.-obsmean,(lenx,leny,lenz),epsilon2;fracindex =Igrid);


In [ ]:
jcut=30
pcolor(xs[:,jcut,:],-zs[:,jcut,:],fi[:,jcut,:].+obsmean),colorbar()

## How to make a horizontal plot



In [ ]:
zcut=zc[1,1,kcut]
# Create fake data
sigmaonzlevel=ztosigma.(xs[:,:,1],ys[:,:,1],zcut.+ 0.0 .*ys[:,:,1])


Igridcut=localize_separable_grid((xs[:,:,1],ys[:,:,1],sigmaonzlevel), masks, (xs,ys,sigma), )
Htoz, out, outbbox = sparse_interp(masks, Igridcut, iscyclic)

In [ ]:
Htoz=Htoz*sparse_pack(masks)';

In [ ]:
zlevelsol=Htoz*statevector_pack(s.sv, (fi,))

In [ ]:
soltoplot=reshape(zlevelsol,size(xs)[1:2])
# Need to apply mask because that is not included in the interpolation
soltoplot[zcut.>b].=NaN
soltoplot[mask_edit[:,:,1].==0].=NaN
pcolor(xs[:,:,1],ys[:,:,1],soltoplot.+obsmean)

In [ ]:
# Surface
pcolor(xs[:,:,1],ys[:,:,1],fi[:,:,1].+obsmean)